In [1]:
import sys, glob, os
from pathlib import Path
import numpy as np
from multiprocessing import Pool


In [2]:
n=16
cwd= os.getcwd()
prep_train_data_name="ru_small_id.txt"

In [3]:
%cd data
!split -n l/{n} {prep_train_data_name} chunk_
%cd {cwd}

/project/cl-work2/armin-sa/resources/cc100-rus
/home/is/armin-sa/Projects/lm


In [4]:
def binarize_chunk(f_in_path):
    with open(f_in_path, "r") as f_in, open("data/prep_" + Path(f_in_path).stem, "wb") as f_out:
        for line in f_in:
            arr = np.fromstring(line, dtype=np.uint16, sep=" ")
            f_out.write(
                len(arr).to_bytes(2, byteorder=sys.byteorder, signed=False) + arr.tobytes()
            )


In [5]:
with Pool(processes=n) as p:
    p.map(binarize_chunk, glob.glob("data/chunk_*"))

In [6]:
%cd data
!rm chunk_*
!cat prep_chunk_* > ru_small.bin
!rm prep_chunk_*
%cd {cwd}

/project/cl-work2/armin-sa/resources/cc100-rus
/home/is/armin-sa/Projects/lm


In [7]:
# with open("/home/is/armin-sa/Projects/lm/data/ru_small_id.txt") as f_in, open(
#     "/home/is/armin-sa/Projects/lm/data/ru_small.npy", "wb"
# ) as f_out:
#     for line in f_in:
#         arr = np.fromstring(line, dtype=np.uint16, sep=" ")
#         f_out.write(
#             len(arr).to_bytes(16, byteorder=sys.byteorder, signed=False) + arr.tobytes()
#         )
# print(arr)


In [10]:
with open("data/ru_small.bin", "rb") as f_in:
    while count := int.from_bytes(f_in.read(2), byteorder=sys.byteorder, signed=False):
        arr = np.frombuffer(f_in.read(count * 2), count=count, dtype=np.uint16)
        l.append(arr)


In [15]:
print(l[-3])


[    1 22362   974    10   160    59  8430     8  2873     4    66     8
    23    41 11047  7146     5     2]
